## import de librerias

In [2]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

dataframes

In [3]:
df_users = pd.read_csv("recursos/users-details-2023.csv")
df_scores = pd.read_csv("recursos/users-score-2023.csv")
df_anime = pd.read_csv("recursos/anime-dataset-2023.csv")

In [4]:
# Función para calcular la edad a partir de la fecha de nacimiento
def calculate_age(birthday):
    try:
        birth_date = datetime.strptime(birthday.split('T')[0], '%Y-%m-%d')
        today = datetime.now()
        age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
        return age
    except:
        return None

In [5]:
# Procesar la edad
df_users['age'] = df_users['Birthday'].apply(calculate_age)

# Limpiar datos - eliminar edades no razonables y valores nulos
df_users = df_users[(df_users['age'] >= 10) & (df_users['age'] <= 80)]
df_users = df_users.dropna(subset=['Gender', 'age'])

In [6]:
df_users['watched_time'] = (df_users['Episodes Watched'] * 0.366666).round(2)

In [7]:
# Definir los bins (rangos) y etiquetas para grupos de 10 en 10
bins = range(10, 81, 5)  # De 10 a 80 en pasos de 5
labels = [f"{i}-{i+4}" for i in range(10, 80, 5)]  # Ej: "10-14", "15-19", etc.


# Crear una nueva columna con los rangos de edad
df_users['age_group'] = pd.cut(df_users['age'], bins=bins, labels=labels, right=False)

In [8]:
avg_watched_time = df_users.groupby('age_group')['watched_time'].mean().reset_index()
avg_watched_time.columns = ['age_group', 'avg_hours']

C:\Users\tomas\AppData\Local\Temp\ipykernel_19652\1779946765.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_watched_time = df_users.groupby('age_group')['watched_time'].mean().reset_index()


In [11]:
avg_watched_time = df_users.groupby('age_group')['watched_time'].mean().reset_index()

# Crear gráfico de lollipop verdadero
fig = go.Figure()

# Añadir las líneas (los "palos" del lollipop)
for i in range(len(avg_watched_time)):
    fig.add_shape(
        type='line',
        x0=avg_watched_time['age_group'][i],
        y0=0,
        x1=avg_watched_time['age_group'][i],
        y1=avg_watched_time['watched_time'][i],
        line=dict(color='rgb(102, 197, 204)', width=2)
    )

# Añadir los puntos (los "dulces" del lollipop)
fig.add_trace(go.Scatter(
    x=avg_watched_time['age_group'],
    y=avg_watched_time['watched_time'],
    mode='markers',
    marker=dict(color='rgb(102, 197, 204)', size=12),
    name='Horas vistas'
))

# Personalizar el gráfico
fig.update_layout(
    title='Horas Vistas por Grupo de Edad',
    xaxis_title='Grupo de Edad',
    yaxis_title='Horas Vistas Promedio',
    showlegend=False,
    template='plotly_white',
    yaxis=dict(range=[0, avg_watched_time['watched_time'].max() * 1.1])  # Ajustar el rango del eje Y
)

fig.show()

C:\Users\tomas\AppData\Local\Temp\ipykernel_19652\1924453123.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [10]:
# Preparar los datos para el gráfico de radar
radar_dropped = df_users.groupby('age_group')['Dropped'].mean().reset_index()

# Crear gráfico de radar
fig = px.line_polar(
    radar_dropped, 
    r='Dropped', 
    theta='age_group',
    line_close=True,
    title="Promedio de Animes Dropeados por Grupo de Edad",
    color_discrete_sequence=['rgb(220, 176, 242)']
)
fig.update_traces(fill='toself')
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, radar_dropped['Dropped'].max() * 1.2]  # Ajustar rango
        )),
    showlegend=False
)
fig.show()

C:\Users\tomas\AppData\Local\Temp\ipykernel_23704\1367770148.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

